In [15]:
pip install -U scikit-learn 

  Using cached scikit_learn-0.24.1-cp38-cp38-win_amd64.whl (6.9 MB)

  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached scipy-1.6.2-cp38-cp38-win_amd64.whl (32.7 MB)


In [20]:
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans

In [17]:
df = pd.read_csv("fin_vul.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203928 entries, 0 to 203927
Data columns (total 40 columns):
 #   Column                                         Non-Null Count   Dtype
---  ------                                         --------------   -----
 0   Excellent                                      203928 non-null  int64
 1   Fair                                           203928 non-null  int64
 2   Good                                           203928 non-null  int64
 3   Poor                                           203928 non-null  int64
 4   Employee: Full-time                            203928 non-null  int64
 5   Employee: Part-time                            203928 non-null  int64
 6   Full-time student                              203928 non-null  int64
 7   Homemaker                                      203928 non-null  int64
 8   Other                                          203928 non-null  int64
 9   Retired                                        203928 non-n

In [45]:
kmeans = KMeans(n_clusters = 6, init = 'k-means++', random_state = 40)
y_kmeans = kmeans.fit_predict(df)

In [46]:
# when k = 6

cluster_1 = df[y_kmeans == 0]
# working women with fair credit

cluster_2 = df[y_kmeans == 1]
# retired 55+ married women with good credit

cluster_3 = df[y_kmeans == 2]
# working and self-employed women with excellent credit 

cluster_4 = df[y_kmeans == 3]
# working women with few qualifications and good credit

cluster_5 = df[y_kmeans == 4]
# single 30+ working women with excellent credit

cluster_6 = df[y_kmeans == 5]
# single retired women with excellent credit

# cluster_1[['Self-employed', 'Unemployed',
#        'Divorced / dissolved civil partnership', 'Married / civil partnership',
#        'Separated', 'Single', 'Widowed / surviving civil partner', '18-20',
#        '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '55+',
#        'Level 1', 'Level 2', 'Level 3', 'Level 4 or above']].head(20)


In [50]:
pd.set_option('display.max_columns', None)
print(cluster_6.head(10))

    Excellent  Fair  Good  Poor  Employee: Full-time  Employee: Part-time  \
3           1     0     0     0                    0                    0   
11          1     0     0     0                    0                    0   
25          1     0     0     0                    0                    0   
30          1     0     0     0                    0                    0   
40          1     0     0     0                    0                    0   
41          1     0     0     0                    0                    0   
45          1     0     0     0                    0                    0   
46          1     0     0     0                    0                    0   
49          1     0     0     0                    0                    0   
57          1     0     0     0                    0                    0   

    Full-time student  Homemaker  Other  Retired  Self-employed  Unemployed  \
3                   0          0      0        1              0          

In [51]:
# adding a new column representing financial risk 'y', set to 1 for cluster 1 and set to 0 for all other clusters

# Adding column y set to 1
cluster_6['financial-risk'] = 1

# Adding column y set to 0
clusters = [cluster_1, cluster_2, cluster_3, cluster_4, cluster_5]
for dataframe in clusters:
    dataframe['financial-risk'] = 0
    
# concatenating all the dfs
all_clusters = [cluster_1, cluster_2, cluster_3, cluster_4, cluster_5, cluster_6]
df_merge = pd.concat(all_clusters, join='outer', axis=0)

df_merge = df_merge.sort_index()
df_merge.info()

<ipython-input-51-7af8d61bf777>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_6['financial-risk'] = 1
<ipython-input-51-7af8d61bf777>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['financial-risk'] = 0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 203928 entries, 0 to 203927
Data columns (total 41 columns):
 #   Column                                         Non-Null Count   Dtype
---  ------                                         --------------   -----
 0   Excellent                                      203928 non-null  int64
 1   Fair                                           203928 non-null  int64
 2   Good                                           203928 non-null  int64
 3   Poor                                           203928 non-null  int64
 4   Employee: Full-time                            203928 non-null  int64
 5   Employee: Part-time                            203928 non-null  int64
 6   Full-time student                              203928 non-null  int64
 7   Homemaker                                      203928 non-null  int64
 8   Other                                          203928 non-null  int64
 9   Retired                                        203928 non-n

## Training the kNN model

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [54]:
X = df_merge.iloc[:, 0:40]  # selecting all rows and all columns except the last column
y = df_merge.iloc[:, 40]  # selecting all rows and the last column

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [56]:
# finding the optimum number of neighbors --> 201 neighbors

import math
math.sqrt(len(y_test))

201.95544062985775

In [57]:
classifier = KNeighborsClassifier(n_neighbors = 201, p=2, metric='euclidean') # p=2, two outcomes-> financial abuse or not

In [58]:
classifier.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=201)

In [60]:
y_prediction = classifier.predict(X_test)
print(y_prediction)

[0 0 0 ... 0 0 1]


In [61]:
# model evaluation
con_matrix = confusion_matrix(y_test, y_prediction)
print(f1_score(y_test, y_prediction))

0.98559670781893


In [62]:
# confusion matrix represented as a heatmap
plt.rcParams['figure.figsize'] = (15.0, 10.0)
sns.heatmap(con_matrix, annot=True, fmt='d')

NameError: name 'plt' is not defined